In [ ]:
import numpy as np
import tensorflow as tf

import magenta.music as mm
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel

In [ ]:
model_name_melody_2bar  = 'cat-mel_2bar_big'
model_name_melody_16bar = 'hierdec-mel_16bar'
model_name_trio_16bar   = 'hierdec-trio_16bar'

model_name = model_name_trio_16bar
config = configs.CONFIG_MAP[model_name]

model = TrainedModel(config,
                     batch_size=16,
                     checkpoint_dir_or_path='./models/pretrained/{}.ckpt'.format(model_name))

In [ ]:
config.data_converter

In [ ]:
from magenta.protobuf import music_pb2

twinkle_twinkle = music_pb2.NoteSequence()

# Add the notes to the sequence.
twinkle_twinkle.notes.add(pitch=60, start_time=0.0, end_time=0.5, velocity=80)
twinkle_twinkle.notes.add(pitch=60, start_time=0.5, end_time=1.0, velocity=80)
twinkle_twinkle.notes.add(pitch=67, start_time=1.0, end_time=1.5, velocity=80)
twinkle_twinkle.notes.add(pitch=67, start_time=1.5, end_time=2.0, velocity=80)
twinkle_twinkle.notes.add(pitch=69, start_time=2.0, end_time=2.5, velocity=80)
twinkle_twinkle.notes.add(pitch=69, start_time=2.5, end_time=3.0, velocity=80)
twinkle_twinkle.notes.add(pitch=67, start_time=3.0, end_time=4.0, velocity=80)
twinkle_twinkle.notes.add(pitch=65, start_time=4.0, end_time=4.5, velocity=80)
twinkle_twinkle.notes.add(pitch=65, start_time=4.5, end_time=5.0, velocity=80)
twinkle_twinkle.notes.add(pitch=64, start_time=5.0, end_time=5.5, velocity=80)
twinkle_twinkle.notes.add(pitch=64, start_time=5.5, end_time=6.0, velocity=80)
twinkle_twinkle.notes.add(pitch=62, start_time=6.0, end_time=6.5, velocity=80)
twinkle_twinkle.notes.add(pitch=62, start_time=6.5, end_time=7.0, velocity=80)
twinkle_twinkle.notes.add(pitch=60, start_time=7.0, end_time=8.0, velocity=80) 

twinkle_twinkle.total_time = 128

twinkle_twinkle.tempos.add(qpm=60);

mm.plot_sequence(twinkle_twinkle)
mm.play_sequence(twinkle_twinkle, synth=mm.fluidsynth)

In [ ]:
tones = music_pb2.NoteSequence()

tot = 64

for i in range(tot):
    tones.notes.add(pitch=60, start_time=i, end_time=i + 0.9, velocity=80)

tones.total_time = tot

tones.tempos.add(qpm=60);

mm.plot_sequence(tones)
mm.play_sequence(tones)

In [ ]:
with open('./data/lmd_clean/The Beatles/Help!.2.mid', 'rb') as midi_file:
    song_seq = mm.midi_to_sequence_proto(midi_file.read())
    
mm.plot_sequence(song_seq)
mm.play_sequence(song_seq, synth=mm.fluidsynth)

In [ ]:
song_seq

In [ ]:
tensors = config.data_converter.to_tensors(song_seq)
tensors

In [ ]:
config.data_converter

In [ ]:
tensors[0][0]

In [ ]:
extracted_trios = \
      config.data_converter.to_notesequences(
          config.data_converter.to_tensors(song_seq)[1])

In [ ]:
sample = extracted_trios[0]

melody_only = music_pb2.NoteSequence()
melody_only.ticks_per_quarter = sample.ticks_per_quarter
melody_only.tempos.add(qpm=sample.tempos[0].qpm)
melody_only.total_time = sample.total_time

from copy import deepcopy
sample2 = deepcopy(sample)

new_notes = list(filter(lambda note: not note.instrument, sample.notes[:]))
# sample.notes[:] = new_notes
del sample2.notes[:]
# for note in new_notes:
sample2.notes.extend(new_notes)
# for note in sample.notes:
#     if note.instrument == 0:
# #         melody_only.notes.extend([note])

In [ ]:
mm.plot_sequence(sample)
mm.play_sequence(sample, synth=mm.fluidsynth)

In [ ]:
mm.plot_sequence(sample2)
mm.play_sequence(sample2, synth=mm.fluidsynth)

In [ ]:
mm.plot_sequence(melody_only)
mm.play_sequence(melody_only, synth=mm.fluidsynth)

In [ ]:
help(model)

In [ ]:
# inp_tensors = model._config.data_converter.to_tensors(song_seq)
# inp_tensors.inputs
config.data_converter._gap_bars = 999
config.data_converter._to_tensors(melody_only)

In [ ]:
len(extracted_trios[0].notes)

In [ ]:
config

In [ ]:
mm.plot_sequence(extracted_trios[0])
mm.play_sequence(extracted_trios[0], synth=mm.fluidsynth)

In [ ]:
song_seq.total_time

In [ ]:
melody_only = copy()

In [ ]:
z, mu, sigma = model.encode([extracted_trios[0]])
# z, mu, sigma = model.encode_tensors([inp_tensors[0]], lengths=64)
# z, mu, sigma = model.encode([melody_only])

z = z[0]
mu = mu[0]
sigma = sigma[0]

In [ ]:
z.shape

In [ ]:
z.shape

In [ ]:
out_seq = model.decode([z], length=64)[0]
mm.plot_sequence(out_seq)
mm.play_sequence(out_seq, synth=mm.fluidsynth)

In [ ]:
out_seq.total_time